In [ ]:
import os
def get_file_dir(b,i,j,k,p=10):
    return(f'spinodal_model_dat_{b:0.1f}/Simulation/{i:0.2f}_{j:0.2f}_{k:0.2f}_{p:d}')

keys = [[1.0,0.3,0.3,0.4],
        [1.0,0.5,0.0,0.0],
        [1.0,0.5,0.5,0.0],
]
for k in keys:
    f = get_file_dir(*k)
    f = '/scratch/jt35560/codebase/prj3/'+f
    cont = '.x.comp.dump'
    files = []
    files.append(f)
    files.append(f+cont)
    files.append(f+cont.replace('x','y'))
    files.append(f+cont.replace('x','z'))
    
    for fi in files:
        # print(f'scp -vr jt35560@sapelo2.gacrc.uga.edu:{fi} .') # change to your username
        print(fi)
    
    

In [2]:
from ovito.io import import_file
import numpy as np
import vtk


def compile_dat_dump(data_file,dump_file):
    
    vtk_head = f'''# vtk DataFile Version 1.0
{dump_file.split('/')[-1].split('.c')[0]}
ASCII

DATASET UNSTRUCTURED_GRID
''' 

    frame_index = int(data_file.split('/')[-1].split('.')[-2])
    # print(frame_index)
    pipline = import_file(dump_file)
    frames = pipline.source.num_frames
    fram = np.linspace(0,frames,26,dtype=int)
    # print(fram)
    dat_file = import_file(data_file)  
    dat = dat_file.compute()
    bonds = np.array(dat.particles.bonds.topology)
    # print(list(dat.particles.keys()))
    # return 0
    from tqdm import tqdm
    faces = []
    for i,j in tqdm(bonds):
        i_nei = np.unique(bonds[np.where(bonds == i)[0]].flatten())
        i_nei = i_nei[i_nei != i]
        j_nei = np.unique(bonds[np.where(bonds == j)[0]].flatten())
        j_nei = j_nei[j_nei != j]
        
        ij_nei = np.intersect1d(i_nei,j_nei)
        for k in ij_nei:
            faces.append([i,j,k])
            
    faces = np.sort(np.array(faces),axis=-1)
    faces = np.unique(faces,axis=0)
    faces = faces # in data file the bond is using id instead the index in the list
    
    pos_dat = dat.particles.positions.array
    dat_pdimage = dat.particles['Periodic Image'].array
    cell = np.array(dat.cell)
    # print(cell)
    cell_size = np.array([cell[0,0],cell[1,1],cell[2,2]])
    
    pos_dat = pos_dat + dat_pdimage*cell_size
    

    time = fram[frame_index]
    pip_dump = pipline.compute(time)
    
    # pos = pip_dump.particles.positions.array
    pos = pos_dat
    c_1 = pip_dump.particles['c_1[1]'].array
    c_2 = pip_dump.particles['c_1[2]'].array
    c_3 = pip_dump.particles['c_1[3]'].array
    c_name = 'stress'
    field = np.array([c_1,c_2,c_3]).T
    # pd_image = pip_dump.particles['Periodic Image'].array
    # cell = pip_dump.cell.array
    # cell_size = np.array([cell[0,0],cell[1,1],cell[2,2]])
    
    
    

    points = vtk.vtkPoints()
    for i,x in enumerate(pos):
        points.InsertPoint(i, x)
        
        
    ugrid = vtk.vtkUnstructuredGrid()
    ugrid.SetPoints(points)
    
    ugrid.AllocateExact(len(faces),len(faces)*4)
    for _f in faces:
        ugrid.InsertNextCell(vtk.VTK_TRIANGLE, 3, _f)
    
    
    vectors = vtk.vtkDoubleArray()
    vectors.SetNumberOfComponents(3)
    vectors.SetName(c_name)
    for _s in field:
        vectors.InsertNextTuple(_s)
    
    ugrid.GetPointData().SetVectors(vectors)
    
    # Write the polydata to a VTK file
    f_name = data_file[:-4]+f'.{time}.vtk'
    writer = vtk.vtkUnstructuredGridWriter()
    writer.SetFileName(f_name)
    writer.SetInputData(ugrid)
    writer.Write()
    
    # # write stl
    # f_name = dump_file.split('.c')[0]+f'.{time}.stl'
    # with open(f_name,'w') as f:
    #     f.write('solid\n')
    #     for _f in faces:
    #         f.write(f'facet normal 0 0 0\n')
    #         f.write(f'outer loop\n')
    #         for i in _f:
    #             f.write(f'vertex {pos[i][0]:0.2f} {pos[i][1]:0.2f} {pos[i][2]:0.2f}\n')
    #         f.write(f'endloop\n')
    #         f.write(f'endfacet\n')
    #     f.write('endsolid\n')
        
            

    

In [2]:
# dat_file = 'sim_dump/0.30_0.30_0.40_10/x.comp.load.1.dat'
# dump_file = 'sim_dump/0.30_0.30_0.40_10.x.comp.dump'
import os
for root, dirs, files in os.walk('sim_dump'):
    for  f in files:
        if f.endswith('.dat'):
            dat_file = os.path.join(root,f)
            ori = f[0]
            dump_file = dat_file.split(f'/{ori}.')[0]+ f'.{ori}.comp.dump'
            print(dat_file)
            print(dump_file)
            compile_dat_dump(dat_file,dump_file)

sim_dump/0.50_0.00_0.00_10/x.comp.load.12.dat
sim_dump/0.50_0.00_0.00_10.x.comp.dump


100%|██████████| 58230/58230 [00:34<00:00, 1686.03it/s]


sim_dump/0.50_0.00_0.00_10/x.comp.load.6.dat
sim_dump/0.50_0.00_0.00_10.x.comp.dump


100%|██████████| 58230/58230 [00:34<00:00, 1675.72it/s]


sim_dump/0.50_0.00_0.00_10/x.comp.load.7.dat
sim_dump/0.50_0.00_0.00_10.x.comp.dump


...

100%|██████████| 58381/58381 [00:32<00:00, 1812.03it/s]


sim_dump/0.50_0.50_0.00_10/z.comp.load.3.dat
sim_dump/0.50_0.50_0.00_10.z.comp.dump


100%|██████████| 58381/58381 [00:33<00:00, 1719.51it/s]


sim_dump/0.50_0.50_0.00_10/x.comp.load.23.dat
sim_dump/0.50_0.50_0.00_10.x.comp.dump


100%|██████████| 58381/58381 [00:32<00:00, 1774.60it/s]


In [3]:
# dat_file = 'sim_dump/0.30_0.30_0.40_10/x.comp.load.1.dat'
# dump_file = 'sim_dump/0.30_0.30_0.40_10.x.comp.dump'
dat_file = 'sim_dump/0.50_0.00_0.00_10/y.comp.load.14.dat'
dump_file = 'sim_dump/0.50_0.00_0.00_10.y.comp.dump'
compile_dat_dump(dat_file,dump_file)

100%|██████████| 58230/58230 [00:32<00:00, 1788.46it/s]
